# Tagging del Dataset
Se usa un modelo de lenguaje para clasificar manualmente cada conversación. 

In [14]:
from parser import TicketParser
from llm_service import LLMService
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


#Cargando los datos
parser = TicketParser("/Users/javier/Downloads/RETO TEC 2/Tickets de atencion a socios (1).json", "/Users/javier/Downloads/RETO TEC 2/Tickets de atencion a socios (2).json")
LLM = LLMService('apikey')

In [9]:
prompt =  "Se te presenta una conversación entre un cliente o socio y un agente de servicio al cliente. Debes etiquetar la conversacion con un tema que englobe que problema reporta el cliente, solo devuelve el tema como respuesta, no escribas nada mas. "

In [10]:
conversacion = parser.get_text_conversation(0).get("conversation")
print(conversacion)

Eduardo Rodriguez:
Please find the attachments below

Diego Rodriguez:
[#3061]
Mande

Eduardo Rodriguez:
Oye estoy en la embajada

Eduardo Rodriguez:
Una señora se fue pero con la aplicación bloqueada

Diego Rodriguez:
[#3061]
Logro pagar? Y otra pregunta la terminal esta funcionando con datos o WiFi? 

Eduardo Rodriguez:
Si si pago pero no se refleja aquí en la terminal por qué la tenia bloqueada

Eduardo Rodriguez:
Please find the attachments below

Eduardo Rodriguez:
Wfi

Eduardo Rodriguez:
La app

Eduardo Rodriguez:
Nose si se le pueda marcar

Diego Rodriguez:
[#3061]
No, ahorita no se le puede marcar, pero si le enviaré mensaje ya que ella si se comunico con nosotros y le respondí 

Eduardo Rodriguez:
Bueno entonces qué así se quede

Eduardo Rodriguez:
??

Eduardo Rodriguez:
Y ya lo arreglan ustedes

Eduardo Rodriguez:
Mientras que mete la cuenta

Eduardo Rodriguez:
??

Diego Rodriguez:
[#3061]
No se queda así ya tenemos una serie de pasos que seguir en estas Incidencias, estaremo

In [11]:
respuesta = LLM.chat_message([{"role": "user", "content": prompt + conversacion}])

2025-06-04 14:50:20,099 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


In [12]:
respuesta

'Problemas con pagos y bloqueo de la aplicación'

In [15]:
# Obtener el tema de todas las conversaciones
import pandas as pd

data = []

# Obtener el tema de todas las conversaciones
for index, (ticket_id, ticket_data) in enumerate(parser.parsed_tickets.items()):
    try:
        conversacion = ticket_data.get("conversation")
        # Convert the conversation list to a string
        conversacion_str = "\n".join([f"{entry['name']}: {entry['message']}" for entry in conversacion])
        respuesta = LLM.chat_message([{"role": "user", "content": prompt + conversacion_str}])
        data.append({"ticket_id": ticket_id, "respuesta": respuesta})
        logging.info(f"Processed ticket index: {index}, ticket_id: {ticket_id}")
    except Exception as e:
        logging.error(f"Error processing ticket index: {index}, ticket_id: {ticket_id}, error: {e}")
        print(data)
df = pd.DataFrame(data)

2025-06-04 14:54:00,681 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-04 14:54:00,774 - INFO - Processed ticket index: 0, ticket_id: 150156525929
2025-06-04 14:54:01,198 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-04 14:54:01,349 - INFO - Processed ticket index: 1, ticket_id: 150156528181
2025-06-04 14:54:01,873 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-04 14:54:01,959 - INFO - Processed ticket index: 2, ticket_id: 150156564239
2025-06-04 14:54:02,525 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-04 14:54:02,563 - INFO - Processed ticket index: 3, ticket_id: 150156565768
2025-06-04 14:54:03,081 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-04 14:54:03,208 - INFO - Processed ticket index: 4, ticket_id: 150156591201


In [20]:
df['respuesta'].head(10)

0    Problemas con transacciones bloqueadas en la t...
1    Problemas con promociones y cashback en la pla...
2              Problemas en la atención o comunicación
3               Agradecimiento por atención al cliente
4        Problemas con descuentos y condiciones de uso
5           Problema con soporte o atención al cliente
6                 Problemas técnicos con la aplicación
7      Consulta sobre descuentos y uso en restaurantes
8    Problemas con aceptación de beneficios en esta...
9           Dificultad en realizar un pedido en la app
Name: respuesta, dtype: object